# Phase 1: Classification (assign Utility, Application, or Entity Tag)

## 1.1 Create Embeddings

In [ ]:
version = "v_team" #TODO changer
system = "pos" #TODO changer

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import csv
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import csv
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
import ast
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AlbertTokenizer, AlbertModel


In [ ]:
model_type = "albert"

if (model_type == "codebert"): 
    tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base",force_download=False)
    model = AutoModel.from_pretrained("microsoft/codebert-base",force_download=False)
elif (model_type == "hugging-face"):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") 
    model = BertModel.from_pretrained("bert-base-uncased") 
elif (model_type == "roberta"):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = RobertaModel.from_pretrained("roberta-base")
elif (model_type == "gpt2"): 
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")
elif (model_type == "albert"): 
    # pip3 install sentencepiece
    tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
    model = AlbertModel.from_pretrained("albert-base-v2")

    

In [ ]:
# Initialize data object with ground truths
applicationGtPath = "ground_truths/" + version + "/" + system + "/classes/application.txt"
utilityGtPath = "ground_truths/" + version + "/" + system + "/classes/utility.txt"
entityGtPath = "ground_truths/" + version + "/" + system + "/classes/entity.txt"

entity=[]
application=[]
utility=[]
data={}
inp = open(applicationGtPath, "r")
application = inp.read().splitlines()
inp.close()
for c in application:
    data[c]=["application"]
inp = open(utilityGtPath, "r")
utility = inp.read().splitlines()
inp.close()
for c in utility:
    data[c]=["utility"]
inp = open(entityGtPath, "r")
entity = inp.read().splitlines()
inp.close()
for c in entity:
    data[c]=["entity"]

print(data)

In [ ]:
# Loop through source code in path specified and add embeddings in data object
path = './src_code/' + system + '/src_code_formatted/'

for elem in data:
    n=0
    allembadding=[]
    with open(path+elem.lower()+'.java',encoding = "ISO-8859-1",errors="ignore") as f:
        print(elem)
        lines=f.read()
        code_tokenss = tokenizer.tokenize(lines)
        while(len(code_tokenss)>(n)*510):
            code_tokens=code_tokenss[(n)*510:(n+1)*510]
            
            if (model_type == "gpt2"):
                tokens = code_tokens

            else:
                tokens = [tokenizer.cls_token] +  code_tokens + [tokenizer.sep_token]

            tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
            context_embeddings = model(torch.tensor(tokens_ids)[None, :])[0]
            embaddings= context_embeddings[0][0].tolist()
            allembadding.append(embaddings)
            n=n+1
        npmatrix=np.matrix(allembadding)
        meanmatrix=npmatrix.mean(0)
        arr=np.array(meanmatrix[0]).flatten()
        data[elem].append(arr.tolist())


In [ ]:
# Write embeddings to csv file
fileName = version + "_" + system+ "_" + model_type + "_"+ 'embeddings.csv'
with open(fileName, mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for n in data:
        employee_writer.writerow([n,data[n][0], data[n][1]])

## 1.2 Train ML models

In [ ]:
import sys
csv.field_size_limit(sys.maxsize)

X=[]
y=[]
Xpos=[]
ypos=[]
namespos=[]
names=[]

fileName = version + "_" + system+ "_" + model_type + "_"+ 'embeddings.csv'
with open(fileName, newline='') as classEmbaddings:
    reader = csv.reader(classEmbaddings, delimiter=';')
    for row in reader:
        X.append(ast.literal_eval(row[2]))
        y.append(row[1])
        names.append(row[0])

X_train, X_test, y_trainnames, y_testnames = train_test_split(X, names, random_state = 0,test_size=0.7)
y_train=[]
y_test=[]
for yt in y_trainnames:
    y_train.append(y[names.index(yt)])
for yt in y_testnames:
    y_test.append(y[names.index(yt)])
utilityyes=False
for yitem in y_train:
    if(yitem=="utility"):
        utilityyes=True
if not utilityyes:
    for i,yitem in enumerate(y):
        if(yitem=="utility"):
            y_train.append("utility")
            X_train.append(X[i])
            break

results_file = f"{version}_{system}_{model_type}_results.csv"


## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)
cm = confusion_matrix(y_test, dtree_predictions)

from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, dtree_predictions)

## SVM

In [ ]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 2).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)
# model accuracy for X_test  
accuracysvm = svm_model_linear.score(X_test, y_test)
  
# creating a confusion matrix
cmsvm = confusion_matrix(y_test, svm_predictions)
print(accuracysvm)
print(cmsvm)
print(classification_report(y_test, svm_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))





## KNN

In [ ]:
# training a KNN classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)
  
# accuracy on X_test
accuracyknn = knn.score(X_test, y_test)
print (accuracyknn)
  
# creating a confusion matrix
knn_predictions = knn.predict(X_test) 
cmknn = confusion_matrix(y_test, knn_predictions)
print(cmknn)
print(classification_report(y_test, knn_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

## LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf_predictions = clf.predict(X_test)
cmclf = confusion_matrix(y_test, clf_predictions)
# accuracy on X_test
accuracycmclf = clf.score(X_test, y_test)
print (accuracycmclf)
print(cmclf)
print(classification_report(y_test, clf_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

## Gaussian NB

In [ ]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train, y_train)
gnb_predictions = gnb.predict(X_test)
  
# accuracy on X_test
gnb_accuracy = gnb.score(X_test, y_test)
  
# creating a confusion matrix
gnb_cm = confusion_matrix(y_test, gnb_predictions)
print(gnb_accuracy)
print(gnb_cm)

print(classification_report(y_test, gnb_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))

with open(results_file, mode='w', newline='') as results_csv:
    writer = csv.writer(results_csv)
    writer.writerow("GaussianNB")
    writer.writerow(["Metric", "Value"])
    writer.writerow(["Accuracy", gnb_accuracy])
    writer.writerow(["Confusion Matrix"])
    for row in gnb_cm:
        writer.writerow(row)
    writer.writerow(["Classification Report"])

print(f"Results saved to {results_file}")